In [1]:
import uproot
import numpy as np
import awkward as ak
import math as m
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep as hep
import os
import numba as nb
import boost_histogram as bh
import yaml
import json
from coffea import hist as chist
from coffea.nanoevents.methods import vector

import sys
import time

from coffea.util import save
from coffea.util import load

In [2]:
def mkdir(path):
    '''
    创建指定的文件夹
    :param path: 文件夹路径，字符串格式
    :return: True(新建成功) or False(文件夹已存在，新建失败)
    '''
    # 引入模块
    import os

    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")

    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)

    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
         # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在')
        return False

In [5]:
lepton = 'muon'#'muon' or 'electron'
isdata = 'mc'#'data' or 'mc'
output_dir = f'newid_fake_rate_hist'
mkdir(output_dir)
if isdata=='mc':
    MC_list = ['WJets','DYJetsToLL_M-10to50','DYJetsToLL_M-50']
else:
    MC_list = [1]
for MC in MC_list:#'WJets','DYJetsToLL_M-10to50','DYJetsToLL_M-50'
    if lepton=='electron' and isdata=='data':
        data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018A-02Apr2020-v1/NANOAOD',
                                '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018B-02Apr2020-v1/NANOAOD',
                                '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018C-02Apr2020-v1/NANOAOD',
                                '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018D-02Apr2020-v1/NANOAOD']
    elif lepton=='muon' and isdata=='data':
        data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018A-02Apr2020-v1/NANOAOD',
                                '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018B-02Apr2020-v1/NANOAOD',
                                '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018C-02Apr2020-v1/NANOAOD',
                                '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018D-02Apr2020-v1/NANOAOD']
    else:
        if isdata=='mc':
            if MC=='WJets':
                data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
            elif MC=='DYJetsToLL_M-50':
                data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
            elif MC=='DYJetsToLL_M-10to50': 
                data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DYJetsToLL_M-10to50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
            else:
                print('wrong MC name')
                data_nanoAOD_dir = ''
        else:
            print('!!!wrong lepton type!!!')
    if isdata=='data':
        array_dir = f'newid_{isdata}_{lepton}_fake_rate'
    else:
        array_dir = f'newid_{isdata}_{lepton}_fake_rate/{MC}'

    ntotal_file = len(os.listdir(array_dir))

    pwd = os.popen('pwd').readlines()[0].replace('\n','')

    nevents_after_cr_filter = 0
    nevents_pass = 0
    
    nevents_real_after_cr_filter = 0
    nevents_real_pass = 0

    ptbins=np.array([10,20,25,30,35,45])
    etabins=np.array([0,1.0,1.5,2.0,2.5])
    fr_hist = chist.Hist("All",
               chist.Bin("x", "abs_eta", etabins),
               chist.Bin("y", "$\mathrm{p_T[GeV]}$", ptbins),
              )
    fr_hist_pass = chist.Hist("Pass",
               chist.Bin("x", "abs_eta", etabins),
               chist.Bin("y", "$\mathrm{p_T[GeV]}$", ptbins),
              )
    if isdata=='mc':
        if MC=='WJets':
            NGW = 70962104.5234375
            cross_section = 61526.7*1000
        elif MC=='DYJetsToLL_M-50':
            NGW = 100108282.765625
            cross_section = 6189.39*1000 
        elif MC=='DYJetsToLL_M-10to50':
            NGW = 39360042.234375
            cross_section = 18610*1000
        else:
            print('wrong MC name')
        if lepton=='muon':
            equivalent_lumi_highpt = 0.04578
            equivalent_lumi_lowpt = 0.00856
        else:
            equivalent_lumi_highpt = 0.03891
            equivalent_lumi_lowpt = 0.00641

        weight_highpt = cross_section*equivalent_lumi_highpt/NGW
        weight_lowpt = cross_section*equivalent_lumi_lowpt/NGW

    for i,rootfile in enumerate(os.listdir(array_dir)):
        print("\r", end="")
        if not os.path.exists(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_array'):
            print('%d/%d wrong'%(i+1, ntotal_file))
        else:
            if isdata=='data':
                try:
                    x_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_array')
                    y_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_array')
                    fr_hist.fill(x=x_array,y=y_array)
                    nevents_after_cr_filter += len(x_array)
                except:
                    pass
                try:
                    x_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_pass_array')
                    y_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_pass_array')
                    fr_hist_pass.fill(x=x_pass_array,y=y_pass_array)
                    nevents_pass += len(x_pass_array)
                except:
                    pass
            else:
                try:
                    x_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_array')
                    y_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_array')
                    gen_weight_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'gen_weight_array')
                    if lepton=='electron':
                        weight_array = gen_weight_array*1*(y_array<25)*weight_lowpt + gen_weight_array*1*(y_array>=25)*weight_highpt
                    else:
                        weight_array = gen_weight_array*1*(y_array<20)*weight_lowpt + gen_weight_array*1*(y_array>=20)*weight_highpt

                    fr_hist.fill(x=x_array,y=y_array,weight=weight_array)
                    nevents_after_cr_filter += len(x_array)
                    nevents_real_after_cr_filter += np.sum(weight_array)
                except:
                    pass
                try:
                    x_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_pass_array')
                    y_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_pass_array')
                    gen_weight_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'gen_weight_pass_array')

                    if lepton=='electron':
                        weight_pass_array = gen_weight_pass_array*1*(y_pass_array<25)*weight_lowpt + gen_weight_pass_array*1*(y_pass_array>=25)*weight_highpt
                    else:
                        weight_pass_array = gen_weight_pass_array*1*(y_pass_array<20)*weight_lowpt + gen_weight_pass_array*1*(y_pass_array>=20)*weight_highpt

                    fr_hist_pass.fill(x=x_pass_array,y=y_pass_array,weight=weight_pass_array)
                    nevents_pass += len(x_pass_array)
                    nevents_real_pass += np.sum(weight_pass_array)
                except:
                    pass
        print("Progress: %d/%d"%(i+1, ntotal_file),end="")
        print("\t(%d,%d)"%(nevents_after_cr_filter,nevents_pass),end="")
        if isdata=='mc':
            print("\t(%.2f,%.2f)"%(nevents_real_after_cr_filter,nevents_real_pass),end="")
        sys.stdout.flush()
    print()
    mkdir(output_dir)
    if isdata=='data':
        save(fr_hist,f"{output_dir}/{isdata}_{lepton}")
        save(fr_hist_pass,f"{output_dir}/{isdata}_{lepton}_pass")
    else:
        save(fr_hist,f"{output_dir}/{isdata}_{lepton}_{MC}")
        save(fr_hist_pass,f"{output_dir}/{isdata}_{lepton}_{MC}_pass")

newid_fake_rate_hist 目录已存在
Progress: 44/44	(131960,115709)	(3260.96,2986.07)
newid_fake_rate_hist 目录已存在
Progress: 27/27	(52246,43256)	(386.08,337.39)
newid_fake_rate_hist 目录已存在
Progress: 57/57	(427764,378331)	(754.63,694.60)
newid_fake_rate_hist 目录已存在


In [28]:
for i in range(1, 101):
    print("\r", end="")
    print("Download progress: {}%: ".format(i), "▋" * (i // 2), end="")
    sys.stdout.flush()
    time.sleep(0.05)

Download progress: 100%:  ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋

In [7]:
weight_highpt

0.0024056867049036573

In [8]:
weight_lowpt

0.0003963107627456295